## Wedge Project Part 2 - Creating a  Sample of Owners

In [ ]:
import os
import re
import random

import pandas as pd
import numpy as np
import pandas_gbq
from random import sample

from google.cloud import bigquery
from google.oauth2 import service_account

### Connect to GBQ

In [17]:

service_path = r"C:\Users\Sam Beighle\Documents\ADA"
service_file = r'\wedge-project-sam-beighle-3df5c228fad7.json'
gbq_proj_id = 'wedge-project-sam-beighle'

# And this should stay the same. 
private_key =service_path + service_file
# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)
# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)
#double check

for item in client.list_datasets() : 
    print(item.full_dataset_id)

wedge-project-sam-beighle:new_dataset_id
wedge-project-sam-beighle:owner_products
wedge-project-sam-beighle:test
wedge-project-sam-beighle:wedge_project
wedge-project-sam-beighle:wedge_project_beighle


In [16]:
service_path = r"C:\Users\Sam Beighle\Documents\ADA"
service_file = r'C:\Users\Sam Beighle\Documents\ADA\wedge-project-sam-beighle-3df5c228fad7.json'

# Load the service account credentials
credentials = service_account.Credentials.from_service_account_file(private_key)

# Create a BigQuery client
client = bigquery.Client(credentials=credentials)

# Get the email address of the service account
service_account_email = credentials.service_account_email

print(f"Using service account: {service_account_email}")

Using service account: big-query@wedge-project-sam-beighle.iam.gserviceaccount.com


### Query and Creating CSV

In [ ]:
#query and creating owner_sample to fill
owner_sample = []

query = """
        SELECT DISTINCT(card_no)
        FROM `wedge-project-sam-beighle.wedge_project_beighle.transArchive_2*`
        WHERE card_no != 3"""

results = client.query(
    query,
)


for row in results :
    owner_sample.append(row.values()[0])




In [ ]:
# I like to check my results as I go
for row in results:
    print(row)


In [ ]:
random.seed(10)
random_owner_sample = random.choices(owner_sample, k=200)

In [ ]:
# Query to fill with random sample - it was a little small, can change to larger if needed in revision
# I could not get this to run with this - dont know why:FROM `wedge-project-sam-beighle.wedge_project_beighle.transArchive_2*`
query = """     
SELECT card_no, *
FROM `wedge-project-sam-beighle.wedge_project_beighle.transArchive_201004_201006_clean`
WHERE card_no in (
"""
query = query +",".join([str(element) for element in random_owner_sample])+")"

In [ ]:
print(query)

In [ ]:
print(random_owner_sample)

In [ ]:
sample__of_owners = client.query(query).to_dataframe()
sample__of_owners.to_csv("wedge_project_owner_sample_final.csv")

In [ ]:
print(query[:100])

### Output to check if worked, card_no as first column

### Print results for you to see. Card_no in first row

In [15]:

import pandas as pd

sample_of_owners = pd.read_csv("wedge_project_owner_sample_final.csv")

print(sample_of_owners.head())


   Unnamed: 0  card_no                   datetime  register_no  emp_no  \
0           0    19375  2010-05-12 15:56:03+00:00            9      51   
1           1    24973  2010-06-11 20:34:33+00:00            7      24   
2           2    12320  2010-04-23 17:00:32+00:00            4      94   
3           3    23521  2010-06-27 11:08:52+00:00            5      34   
4           4    18466  2010-04-12 12:10:55+00:00            6      25   

   trans_no upc  description trans_type trans_subtype  ... batchHeaderID  \
0        13   0       Change          T            CA  ...           NaN   
1        51   0  Credit Card          T            CC  ...           NaN   
2        61   0       Change          T            CA  ...           NaN   
3         2   0  Credit Card          T            CC  ...           NaN   
4        19   0       Change          T            CA  ...           NaN   

   local  organic  display  receipt  card_no_1  store  branch  match_id  \
0      0      NaN      